In [91]:
import pandas as pd
from datetime import date

In [92]:
# global recoding on age and pram on sex 
anon_data_global_recoding_pram = pd.read_excel("anon_data/anon_data_global_recoding_pram.xlsx")
# global recoding on age, global recoding on marital status 
anon_data_global2 = pd.read_excel("anon_data/anon_data_global2.xlsx")
# global recodign on age, global recoding on marital status, and pram on sex 
anon_data_global2_pram = pd.read_excel("anon_data/anon_data_global2_pram.xlsx")

results_data = pd.read_excel("data/public_data_resultsE.xlsx")
pub_data_register = pd.read_excel('data/public_data_registerE.xlsx')

c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


#### Testing if the anonymised datasets can be de-anonymised 

In [93]:
quasi_identifiers1 = ["age_group", "citizenship", "marital_status", "zip"]
quasi_identifiers2 = ["age_group", "citizenship", "marital_status", "zip", "sex"]

def k_anonymity_violations(data, quasi_identifiers, k_levels=[2, 3, 5]):
    # Group by quasi-identifiers to get the counts for each unique combination
    grouped = data.groupby(quasi_identifiers).size().reset_index(name='count')
    
    # Dictionary to store the number of rows violating each k-level
    violations = {}
    total_rows = len(data)

    for k in k_levels:
        # Count rows where the group size is less than k
        violating_rows = grouped[grouped['count'] < k]['count'].sum()
        # Store as a percentage for comparison with R's output
        violations[k] = violating_rows, (violating_rows / total_rows) * 100

    return violations

In [94]:
def identify_k_anonymity_violations(data, quasi_identifiers, k=2):
    # Group by quasi-identifiers to count each unique combination
    grouped = data.groupby(quasi_identifiers).size().reset_index(name='count')
    
    # Filter for combinations where the count is less than k (i.e., violate k-anonymity)
    violating_combinations = grouped[grouped['count'] < k]
    
    # Merge to find the original rows that match these violating combinations
    violations = data.merge(violating_combinations.drop(columns='count'), on=quasi_identifiers, how='inner')
    
    return violations

Global recoding on age, pram on sex

In [95]:
k_anonymity_violations(anon_data_global_recoding_pram, quasi_identifiers1)
vioalting_rows_global1_pram = identify_k_anonymity_violations(anon_data_global_recoding_pram, quasi_identifiers1)

Global recoding on age, global recoding on marital status 

In [96]:
k_anonymity_violations(anon_data_global2, quasi_identifiers1)
vioalting_rows_global2 = identify_k_anonymity_violations(anon_data_global2, quasi_identifiers1)

Global recoding on age, global recoding on marital status, pram on sex

In [97]:
k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)
vioalting_rows_global2_pram = identify_k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)

##### Deanonymising the dataset 

Global recoding on age, global recoding on marital status, pram on sex

In [ ]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
pub_data_register["dob"] = pd.to_datetime(pub_data_register["dob"])
pub_data_register['age'] = pub_data_register['dob'].apply(lambda x: calculate_age(x))

age_bins = [18, 30, 38, 48, 58, 70, 101]  # Adjust as needed
#age_labels = ['0-18', '19-30', '31-45', '46-60', '61+']
pub_data_register['age_group'] = pd.cut(pub_data_register['age'], bins=age_bins, right=True)

# define map 
status_map = {
    'Divorced': 'Not married',
    'Married/separated': 'Married',
    'Never married': 'Not married',
    'Widowed': 'Not married'
}
labels = ["(18,30]", "(30,38]", "(38,48]", "(48,58]", "(58,70]", "(70,101]"] 
# Apply the map to the marital_status column
pub_data_register["marital_status"]= pub_data_register["marital_status"].map(status_map)
#pub_data_register["marital_status"]= pub_data_register["marital_status"].apply(lambda x: status_map(x))
pub_data_register.groupby("age_group").count()

,name,sex,dob,zip,citizenship,marital_status,last_voted,age
age_group,,,,,,,,
"(18, 30]",267,267,267,267,267,267,267,267
"(30, 38]",221,221,221,221,221,221,221,221
"(38, 48]",288,288,288,288,288,288,288,288
"(48, 58]",315,315,315,315,315,315,315,315
"(58, 70]",276,276,276,276,276,276,276,276
"(70, 101]",192,192,192,192,192,192,192,192


In [99]:
matched_df = anon_data_global2_pram.merge(pub_data_register, on=["zip", "marital_status", "citizenship"], how='inner')
matched_df

,Unnamed: 0,sex_x,evote,zip,education,citizenship,marital_status,party,age_group_x,name,sex_y,dob,last_voted,age,age_group_y
0,1,Female,0,2400,Vocational bachelors educations,Denmark,Not married,Red,"(38,48]","Ramirez-Salaz, Jasmine",Female,2003-06-04,1,21,"(18, 30]"
1,1,Female,0,2400,Vocational bachelors educations,Denmark,Not married,Red,"(38,48]","Johnson, Krislynn",Female,2002-10-25,1,22,"(18, 30]"
2,1,Female,0,2400,Vocational bachelors educations,Denmark,Not married,Red,"(38,48]","Nathalia, Annie",Female,2002-07-23,1,22,"(18, 30]"
3,1,Female,0,2400,Vocational bachelors educations,Denmark,Not married,Red,"(38,48]","al-Salameh, Faseeha",Female,1995-10-17,1,29,"(18, 30]"
4,1,Female,0,2400,Vocational bachelors educations,Denmark,Not married,Red,"(38,48]","Lucero, Tanya",Female,1992-07-17,2,32,"(30, 38]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34938,136,Female,0,2300,Vocational Education and Training (VET),Poland,Not married,Red,"(48,58]","al-Hallal, Shafeeqa",Female,1970-04-04,0,54,"(48, 58]"
34939,145,Female,0,2300,Primary education,Afghanistan,Not married,Invalid vote,"(18,30]","Stacks, Kristina",Female,2004-10-18,0,20,"(18, 30]"
34940,145,Female,0,2300,Primary education,Afghanistan,Not married,Invalid vote,"(18,30]","Mendoza, Susana",Female,1986-11-28,2,37,"(30, 38]"
34941,163,Female,1,2100,Primary education,Syria,Not married,Green,"(30,38]","Morrow, Magnolia",Female,1988-03-10,1,36,"(30, 38]"


In [100]:
"(48,58]" == "(48, 58]"

False

In [101]:
quasi_identifiers1

['age_group', 'citizenship', 'marital_status', 'zip']